In [ ]:
#@title Conectar com o Google Drive

from google.colab import drive
drive.mount('/content/drive')

#Conectando o ambiente ao Google Drive

Mounted at /content/drive


In [ ]:
#@title  Puxar Base de Dados


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, date
import statistics
from statistics import StatisticsError
from sklearn.linear_model import LinearRegression


#Importando bibliotecas necessárias

uri = '/content/drive/Shareddrives/Colab_Grupo2/Colab/Data/dados_testev2.csv'

data = pd.read_csv(uri)


#Declarando caminho do conjunto de dados e instanciando um dataframe

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (22,24,25,75,76,77,78,89,90,91,92) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
#@title Algoritmo de tratamento de dados

#@markdown Execute o algoritmo para corrigir a sua base de dados

data_columns = data.columns
data_columns_date = ["dob", "date_last_fu", "dtrecidiva", "data_2"]
data_columns_string_to_numbers = ["rec01", "rec02", "rec03", "rec04", "meta01", "meta02", "meta03", "meta04"]
data_columns_str_to_percentage = ["er_percentage", "progesteron_perct"]

#Delcarando as variáveis de armazenamento de lista de colunas a serem tratadas

incorrect_values = {
    "0": ["neg", "NEG", "inconclusivo"],
    "10": ["1-10"],
    "98": ["980"],
    "100": ["1000"],
    "70": ["20-70","70%"]
}

#Variável de erros de digitação a serem substituidos pelas chaves

for x in range(len(data_columns_str_to_percentage)):
  data[data_columns_str_to_percentage[x]] = data[data_columns_str_to_percentage[x]].astype(str)
  for key, values in incorrect_values.items():
    for especific_values in values:
      data[data_columns_str_to_percentage[x]] = data[data_columns_str_to_percentage[x]].str.replace(especific_values,key)
  data[data_columns_str_to_percentage[x]] = data[data_columns_str_to_percentage[x]].str.lstrip('nan')
  data[data_columns_str_to_percentage[x]] = pd.to_numeric(data[data_columns_str_to_percentage[x]], downcast="integer")

#Esse laço é capaz de corrigir os valores das colunas listadas especialmente por erros de digitação, transformando-os em int


for x in range(len(data_columns_string_to_numbers)):
  data[data_columns_string_to_numbers[x]] = data[data_columns_string_to_numbers[x]].astype(str)
  data[data_columns_string_to_numbers[x]] = data[data_columns_string_to_numbers[x]].str.lstrip('C')
  data[data_columns_string_to_numbers[x]] = data[data_columns_string_to_numbers[x]].str.lstrip('nan')
  data[data_columns_string_to_numbers[x]] = pd.to_numeric(data[data_columns_string_to_numbers[x]], downcast="integer")

#Esse laço é capaz de corrigir os valores das colunas listadas especialmente por erros de digitação, transformando-os em int

for x in range(len(data_columns)):
  data[data_columns[x]].replace(np.inf, np.nan, inplace=True)

#Esse laço é capaz de substituir valores inf por nulos em todo o data frame

for x in range(len(data_columns_date)):
  data[data_columns_date[x]] = pd.to_datetime(data[data_columns_date[x]])

#Esse laço é capaz de transformar strings em forma de datas

#NOVO

def specif_sort(x):
  data_filter_1 = x.loc[x['redcap_repeat_instrument'] != 'dados_antropometricos']
  data_filter_2 = x.loc[x['redcap_repeat_instrument'] == 'dados_antropometricos']
  data_filter_2 = data_filter_2.sort_values(by='data_2')
  return pd.concat([data_filter_1, data_filter_2])

data = data.groupby(by='record_id').apply(specif_sort)

data.reset_index(drop=True, inplace=True)

#Essa função é capaz de ordenar todas as linhas que possuem valor na coluna data_2 por id
#e ainda sim manter esses valores ordenados mesmo para linhas em que data_2 é nula

#data.reset_index(drop=True, inplace=True)




In [ ]:
#@title Algoritmo de exclusão de colunas

#@markdown Informe a coluna a ser deletada

default_columns= True #@param {type:"boolean"}
column = 'partos' #@param ["record_id", "redcap_repeat_instrument", "redcap_repeat_instance", "dob", "escolari", "level_education", "race", "pregnancy_history", "partos", "abortion", "breast_feeding", "breast_feeding_time", "menarche", "period", "bmi", "contraceptive_method___1", "contraceptive_method___2", "contraceptive_method___3", "contraceptive_method___4", "contraceptive_method___999", "contraceptive", "hormone_therapy", "hormone_therapy_tasy", "tempo_rep_hormo", "tempo_rep_hormo_tasy", "antec_fam_cancer_mama", "tobaco", "alcohol", "tobaco_type", "alcohol_type___1", "alcohol_type___2", "alcohol_type___3", "family_cancer", "familial_degree___1", "familial_degree___2", "familial_degree___3", "family_cancer_id___1", "family_cancer_id___2", "family_cancer_id___3", "family_cancer_id___4", "family_cancer_id___5", "family_cancer_id___6", "mother_tumor", "sister_n", "daughter_n", "sister_tumor_1", "daughter_tumor_1", "birads", "benign", "primary_diganosis", "diff_tubular", "nuclear_grade", "mitotic_index", "histological_grade", "grau_hist", "multicentricidade", "multifocal", "tumor_margin", "er_ihc", "pr_ihc", "tumor_subtype", "progesteron_perct", "er_percentage", "her2_ihc", "her2_fish", "ki67_perc", "date_last_fu", "follow_up_days", "ultinfo", "recidive", "dtrecidiva", "follow_up_days_recidive", "reclocal", "recregio", "recdist", "rec01", "rec02", "rec03", "rec04", "trathosp", "topo", "basediag", "desctopo", "morfo", "tumor_stage", "t", "n", "m", "meta01", "meta02", "meta03", "meta04", "tipotrat", "treatment", "chemotherapy", "hormontx", "hormontxtype", "tipo_qt_neo_her2", "radioterapia", "hormone_therapy_yn", "seguim_qt_realizou", "seguim_qt", "data_2", "weight", "height", "gli", "hb", "ht", "leuco", "neutro", "linfocitos", "ureia", "creat", "hiv"]

#Este formulário armazena se o usuário quer utilizar as colunas de exclusão padrão ou 
#selecionar aquela que ele quer excluir

#NOVO

data['data_2_first']=data['data_2'].copy()
data['data_2_last']=data['data_2'].copy()

#Criação de colunas novas de data de consulta item b

default = ["desctopo","descmorfo","hormone_therapy_tasy","tempo_rep_hormo_tasy","redcap_repeat_instrument","redcap_repeat_instance", "dtrecidiva", "data_2"]

for x in data.columns:
  if data[x].notna().sum() < 1:
    default.append(x)

default_data_1 =["dob", "escolari", "level_education", "race", "pregnancy_history", "partos", "abortion", "breast_feeding", "breast_feeding_time", "menarche", "period", "contraceptive_method___1", "contraceptive_method___2", "contraceptive_method___3", "contraceptive_method___4", "contraceptive_method___999", "contraceptive", "hormone_therapy", "tempo_rep_hormo", "antec_fam_cancer_mama", "tobaco", "alcohol", "alcohol_type___1", "alcohol_type___2", "alcohol_type___3", "family_cancer", "familial_degree___1", "familial_degree___2", "familial_degree___3", "family_cancer_id___1", "family_cancer_id___2", "family_cancer_id___3", "family_cancer_id___4", "family_cancer_id___5", "family_cancer_id___6", "sister_n", "daughter_n", "birads", "benign", "primary_diganosis", "nuclear_grade", "mitotic_index", "histological_grade", "grau_hist", "multicentricidade", "multifocal", "tumor_margin", "er_ihc", "pr_ihc", "tumor_subtype", "progesteron_perct", "er_percentage", "her2_ihc", "her2_fish", "ki67_perc", "date_last_fu", "follow_up_days", "ultinfo", "recidive", "follow_up_days_recidive", "reclocal", "recregio", "recdist", "rec01", "rec02", "rec03", "rec04", "trathosp", "topo", "basediag", "morfo", "tumor_stage", "t", "n", "m", "meta01", "meta02", "meta03", "meta04", "treatment", "chemotherapy", "tipo_qt_neo_her2", "radioterapia", "hormone_therapy_yn", "seguim_qt_realizou", "seguim_qt", "data_2_first", "data_2_last"]

#Vetor de colunas que usam método de primeiro valor e ultimo valor

default_data_2 =["weight", "height","bmi", "tobaco_type", "mother_tumor","sister_tumor_1", "daughter_tumor_1", "diff_tubular", "data_2_last"]

#Vetor de colunas que usam método ultimo valor, flutuacao, média, moda e devio pardrão

default_data_3 = ["weight", "height","bmi", "tobaco_type", "mother_tumor","sister_tumor_1", "daughter_tumor_1", "diff_tubular", "dob", "escolari", "level_education", "race", "pregnancy_history", "partos", "abortion", "breast_feeding", "breast_feeding_time", "menarche", "period", "contraceptive_method___1", "contraceptive_method___2", "contraceptive_method___3", "contraceptive_method___4", "contraceptive_method___999", "contraceptive", "hormone_therapy", "tempo_rep_hormo", "antec_fam_cancer_mama", "tobaco", "alcohol", "alcohol_type___1", "alcohol_type___2", "alcohol_type___3", "family_cancer", "familial_degree___1", "familial_degree___2", "familial_degree___3", "family_cancer_id___1", "family_cancer_id___2", "family_cancer_id___3", "family_cancer_id___4", "family_cancer_id___5", "family_cancer_id___6", "sister_n", "daughter_n", "birads", "benign", "primary_diganosis", "nuclear_grade", "mitotic_index", "histological_grade", "grau_hist", "multicentricidade", "multifocal", "tumor_margin", "er_ihc", "pr_ihc", "tumor_subtype", "progesteron_perct", "er_percentage", "her2_ihc", "her2_fish", "ki67_perc", "date_last_fu", "follow_up_days", "ultinfo", "recidive", "follow_up_days_recidive", "reclocal", "recregio", "recdist", "rec01", "rec02", "rec03", "rec04", "trathosp", "topo", "basediag", "morfo", "tumor_stage", "t", "n", "m", "meta01", "meta02", "meta03", "meta04", "treatment", "chemotherapy", "tipo_qt_neo_her2", "radioterapia", "hormone_therapy_yn", "seguim_qt_realizou", "seguim_qt", "data_2_first"]

#Vetor de colunas que usam método primeiro valor, flutuacao, média, moda e devio pardrão

for x in default:
  if x in default_data_1:
    default_data_1.remove(x)
  if x in default_data_2:
    default_data_2.remove(x)
  if x in default_data_3:
    default_data_3.remove(x)


#Lista de colunas padrões a serem excluidas

if default_columns == True:
  data = data.drop(columns = default)

  data_1 = data.copy()
  data_2 = data.copy()
  data_3 = data.copy()

  data_1 = data_1.drop(columns = default_data_1)
  data_2 = data_2.drop(columns = default_data_2)
  data_3 = data_3.drop(columns = default_data_3)

else:
  del data[column]

#Essa condição verifica se o usuário que usar as colunas padrões, caso verdadeiro, percorre
#a lista de colunas a serem excluídas


In [ ]:
#@title Algoritmo de preenchimento de nulos

default_columns= True #@param {type:"boolean"}

#@markdown Informe o método de preenchimento de valores nulos
method = 'media' #@param ["mediana", "moda", "media",  "primeiro_Registro"]

#@markdown Informe as coluna que você deseja implementar este método
column = 'period' #@param ["record_id", "redcap_repeat_instrument", "redcap_repeat_instance", "dob", "escolari", "level_education", "race", "pregnancy_history", "partos", "abortion", "breast_feeding", "breast_feeding_time", "menarche", "period", "bmi", "contraceptive_method___1", "contraceptive_method___2", "contraceptive_method___3", "contraceptive_method___4", "contraceptive_method___999", "contraceptive", "hormone_therapy", "hormone_therapy_tasy", "tempo_rep_hormo", "tempo_rep_hormo_tasy", "antec_fam_cancer_mama", "tobaco", "alcohol", "tobaco_type", "alcohol_type___1", "alcohol_type___2", "alcohol_type___3", "family_cancer", "familial_degree___1", "familial_degree___2", "familial_degree___3", "family_cancer_id___1", "family_cancer_id___2", "family_cancer_id___3", "family_cancer_id___4", "family_cancer_id___5", "family_cancer_id___6", "mother_tumor", "sister_n", "daughter_n", "sister_tumor_1", "daughter_tumor_1", "birads", "benign", "primary_diganosis", "diff_tubular", "nuclear_grade", "mitotic_index", "histological_grade", "grau_hist", "multicentricidade", "multifocal", "tumor_margin", "er_ihc", "pr_ihc", "tumor_subtype", "progesteron_perct", "er_percentage", "her2_ihc", "her2_fish", "ki67_perc", "date_last_fu", "follow_up_days", "ultinfo", "recidive", "dtrecidiva", "follow_up_days_recidive", "reclocal", "recregio", "recdist", "rec01", "rec02", "rec03", "rec04", "trathosp", "topo", "basediag", "morfo", "tumor_stage", "t", "n", "m", "meta01", "meta02", "meta03", "meta04", "tipotrat", "treatment", "chemotherapy", "hormontx", "hormontxtype", "tipo_qt_neo_her2", "radioterapia", "hormone_therapy_yn", "seguim_qt_realizou", "seguim_qt", "data_2", "weight", "height", "gli", "hb", "ht", "leuco", "neutro", "linfocitos", "ureia", "creat", "hiv"]
 
#Este formulário armazena se o usuário quer utilizar as colunas de preenchimento padrão ou 
#selecionar aquela com o método escolhido

columns = {
    "flutuacao": ["weight"],
    "media": ["height"],
    "moda": ["tobaco_type", "mother_tumor","sister_tumor_1", "daughter_tumor_1", "diff_tubular"],
    "desvio_Pradao": ["bmi"]
}

#Declarando em biblioteca as colunas com seus respectivos métodos

functions = locals()

def func_lregister(mini_data):
    lista_valores = []
    for c in mini_data.columns:
        index_m = mini_data[c].last_valid_index()
        if index_m != None:
            valor_nao_nulo = mini_data[c].loc[index_m]
            lista_valores.append(valor_nao_nulo)
        else:
            lista_valores.append(np.nan)
      
    return pd.DataFrame(data=[lista_valores], columns=mini_data.columns)

def func_fregister(mini_data):
    lista_valores = []
    for c in mini_data.columns:
        index_m = mini_data[c].first_valid_index()
        if index_m != None:
            valor_nao_nulo = mini_data[c].loc[index_m]
            lista_valores.append(valor_nao_nulo)
        else:
            lista_valores.append(np.nan)
      
    return pd.DataFrame(data=[lista_valores], columns=mini_data.columns)

#Essa função utilizada nos metodos de primeiro e ultimo registro e basicamente, identificam os primeiros
#e ultimos valores de cada coluna em realação a cada ID pello groupby da função anterior que chama esta


def flutuacao(column):
  def func(x, column):
    y = x[column].dropna()
    if len(y) > 0:
      x0 = y.iloc[0]
      x1 = y.iloc[-1]
      total = (x1 - x0)/ x0
      return total
    return np.nan
  dfflutuacao = data_1.groupby(by='record_id').apply(lambda x: func(x,column))

  for z in data_1["record_id"]:
    data_1.loc[data_1['record_id'] == z, column] = dfflutuacao.loc[z]
    
  return data_1

# Função que vai realizar a flutuação 

#Essa função substitui os valores da coluna  utilizando o método de mediana dos valores presentes

def moda(column):
  def func(x,column):
    try:
      y = x[column].dropna().to_numpy()
      x[column] = statistics.mode(y)  
    except IndexError:
      pass
    except StatisticsError:
      pass
    return x
  
  return data_1.groupby(by=["record_id"]).apply(lambda x: func(x,column))

#Essa função substitui os valores da coluna  utilizando o método de moda dos valores presentes

def media(column):
  def func(x,column):
    try:
      x[column] = x[column].dropna().mean()
    except IndexError:
      pass
      
    return x

  return data_1.groupby(by=["record_id"]).apply(lambda x: func(x,column))

#Essa função substitui os valores da coluna  utilizando o método de média dos valores presentes

def primeiro_Registro():
  return data_2.groupby(by=['record_id']).apply(func_fregister)

def ultimo_Registro():
  return data_3.groupby(by=['record_id']).apply(func_lregister)

#Essa função substitui os valores da coluna  utilizando o método de primeiro valor presente na coluna

def desvio_Pradao(column):
  def func(x,column):
    try:
      x[column] = x[column].std()
    except IndexError:
      pass
    return x

  return data_1.groupby(by=["record_id"]).apply(lambda x: func(x,column))


if default_columns == True:
  for method, column in columns.items():
    for especif_column in column:
      data_1 = functions[f'{method}'](especif_column)
  data_2 = primeiro_Registro()
  data_3 = ultimo_Registro()
  #NOVO

else:
  data_1 = functions[f'{method}']()

  #Essa condição verifica se o usuário quer executar o preenchimento de colunas padrões,
  #caso verdadeiro, executa laço dentro de laço que percorre os métodos e colunas a serem executadas



In [ ]:
#@title Algoritmo de individualização de IDs

#@markdown Execute o algoritmo para individualizar sua base de dados

data_1.drop_duplicates(subset='record_id', inplace = True)

#Exclusão de IDs duplicados

#NOVO

data_2.reset_index(drop=True, inplace=True)

data_3.reset_index(drop=True, inplace=True)

#Reset do índice dos dataframes

#NOVO

data_1 = data_1.merge(data_2, on="record_id")

#Merge de colunas pelo id

data = data_1.merge(data_3, on="record_id")

#Merge de colunas pelo id

del data['record_id']

#Essa célula executa a deleção de linhas duplicadas pelo valor id e consequentemente deleta a coluna de IDs


In [ ]:
#@title Algoritmo de criação de colunas

#@markdown Execute o algoritmo para criar as colunas nescessárias

#NOVO

today = date.today()
ages_1 = []
ages_2 = []
ages_3 = []

#Declarando variáveis de armazenamento e de data atual

for x in range(len(data)):
  if data["ultinfo"][x] > 2:
    ages_1.append((pd.DatetimeIndex(data['date_last_fu'])[x].year - pd.DatetimeIndex(data['dob'])[x].year - ((pd.DatetimeIndex(data['date_last_fu'])[x].month, pd.DatetimeIndex(data['date_last_fu'])[x].day) < (pd.DatetimeIndex(data['dob'])[x].month, pd.DatetimeIndex(data['dob'])[x].day)))*365.2425)
  else:
    ages_1.append((today.year - pd.DatetimeIndex(data['dob'])[x].year - ((today.month, today.day) < (pd.DatetimeIndex(data['dob'])[x].month, pd.DatetimeIndex(data['dob'])[x].day)))*365.2425)

#Este laço extrai informação de idade da data de nascimento caso eles esteja vivo ou morto para cada id

for x in range(len(data)):
  ages_2.append((pd.DatetimeIndex(data['date_last_fu'])[x].year - pd.DatetimeIndex(data['data_2_first'])[x].year - ((pd.DatetimeIndex(data['date_last_fu'])[x].month, pd.DatetimeIndex(data['date_last_fu'])[x].day) < (pd.DatetimeIndex(data['data_2_first'])[x].month, pd.DatetimeIndex(data['data_2_first'])[x].day)))*365.2425)

#Este laço extrai informação de tempo de tratamento da data de ultima informação para cada id

for x in range(len(data)):
  ages_3.append((pd.DatetimeIndex(data['data_2_last'])[x].year - pd.DatetimeIndex(data['data_2_first'])[x].year - ((pd.DatetimeIndex(data['data_2_last'])[x].month, pd.DatetimeIndex(data['data_2_last'])[x].day) < (pd.DatetimeIndex(data['data_2_last'])[x].month, pd.DatetimeIndex(data['data_2_last'])[x].day)))*365.2425)

#Este laço extrai informação de tempo de realização de exames da ultima e primeira data de consulta para cada id

data['age']=ages_1
data['treatment_time']=ages_2
data['exam_time']=ages_3

#Criação das novas colunas item b

del data['date_last_fu']
del data["dob"]
del data["data_2_first"]
del data["data_2_last"]

#Deleção das coluna não necessárias neste momento

default_columns = ["age", "treatment_time", "exam_time"]

for x in default_columns:
  data.loc[(data[x] < 0),x] = np.nan

#Este laço substitui valores menores que 0 por nulo

data = data.dropna(subset=['follow_up_days'])

#Excluindo os valores nulos na coluna target do modelo

In [ ]:
#@title Método Head com 100 linhas

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

data.head(100)

#Definição de linhas e colunas ilimitadas com a exibição das primeiras 100 linhas

In [ ]:
data.to_csv("/content/drive/Shareddrives/Colab_Grupo2/Colab/Data/dataToPredict.csv", index = False)